In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load data
df = pd.read_csv("../data/survey_data.csv")

# Create charts directory
os.makedirs("../app/static/charts", exist_ok=True)


In [ ]:
# 1. Top Ages by Income
plt.figure(figsize=(12,6))
df.groupby('Age')['Income'].mean().nlargest(10).plot.bar()
plt.title("Top 10 Ages by Average Income")
plt.savefig("../app/static/charts/top_ages.png", bbox_inches='tight')
plt.close()

In [ ]:
# 2. Gender Spending Distribution
gender_spending = df.groupby('Gender')[['Utilities','Entertainment',
                                      'School Fees','Shopping','Healthcare']].mean()
gender_spending.plot(kind='bar', stacked=True, figsize=(14,8))
plt.title("Average Spending by Gender")
plt.savefig("../app/static/charts/gender_spending.png", bbox_inches='tight')
plt.close()